<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-IllinoisGRMHD-NRPyfied: `ADM_3METRIC` and `CONF_METRIC`

## Authors: Leo Werneck & Zach Etienne

<font color='red'>**This module is currently under development**</font>

## In this tutorial notebook we implement the `ADM_3METRIC` and `CONF_METRIC` variables used by the NRPyfied version of `IllinoisGRMHD`

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

0. [Step 0](#loading_modules): **Loading necessary Python/NRPy+ modules**
1. [Step 1](#adm_3metric_files): **`ADM_3METRIC` files**
    1. [Step 1.a](#compute_gamma_inv_and_sqrtgamma): *The `compute__ADM_gammaUU_and_sqrtgamma__in_terms_of__ADM_3METRIC.h` file*
    1. [Step 1.b](#compute_g4dd): *The `compute__g4DD__in_terms_of__ADM_3METRIC` file*
    1. [Step 1.c](#compute_g4uu): *The `compute__g4UU__in_terms_of__ADM_3METRIC` file*
1. [Step 2](#conf_metric_vars): **`CONF_METRIC` files**
    1. [Step 2.a](#compute__lapseinv_psi2_psi4_psi6): *The `compute__lapseinv_psi2_psi4_psi6__in_terms_of__ADM_3METRIC_and_CONF_METRIC.h` file*
1. [Step 3](#tmunu): **Computing the energy-momentum tensor**
    1. [Step 3.a](#tmunu_grhd): *Computing the GRHD energy-momentum tensor*
    1. [Step 3.b](#tmunu_em): *Computing the EM energy-momentum tensor*
    1. [Step 3.c](#tmunu_tupmunu): *The `compute__TUPMUNU.h` file*
    1. [Step 3.d](#tmunu_tdnmunu): *The `compute__TDNMUNU.h` file*
1. [Step n](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='loading_modules'></a>

# Step 0: Loading necessary Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{loading_modules}$$

We now load the necessary Python and NRPy+ modules needed by this tutorial notebook.

In [1]:
# Import Python modules
import os,sys            # Python module: used for system and OS specific commands
import sympy as sp       # Python module: used for symbolic expressions
import re                # Python module: used to manipulate regular expressions

# Register NRPy+ root directory to the path
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Register IGM src directory


# Load NRPy+ modules
from outputC import *        # NRPy+ module: used to output sympy expressions to C
import indexedexp as ixp     # NRPy+ module: used to generate indexed expressions (e.g. g_{\mu\nu})
import cmdline_helper as cmd # NRPy+ module: used for command line features

# Create the NRPy+ header file directory, if it doesn't already exist
IGM_src_dir_path = os.path.join("..","src")
cmd.mkdir(os.path.join(IGM_src_dir_path,"NRPy_generated_headers"))
NRPy_headers_dir_path = os.path.join(IGM_src_dir_path,"NRPy_generated_headers")

# Set up a neat function to output the expressions to NRPy+ generated files
def NRPy_IGM_write_to_file(filepath,filename,contents):
    with open(filepath,"w") as file:
        file.write("""
/* .-----------------------------------------------------------------------.
 * | This file was generated by NRPy+ for IllinoisGRMHD, as documented in: |
 * |        Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.ipynb         |
 * .-----------------------------------------------------------------------.
 * |                Author(s): Leo Werneck and Zach Etienne                |
 * .-----------------------------------------------------------------------.
 * |             Source: https://github.com/leowerneck/NRPyIGM             |
 * .-----------------------------------------------------------------------.
 *
 * File start: """+filename+""" */
\n"""+contents+"""
/* File end  : """+filename+""" */
""")

<a id='adm_3metric_vars'></a>

# Step 1: The `ADM_3METRIC` variable declaration \[Back to [top](#toc)\]
$$\label{adm_3metric_vars}$$

The `ADM_3METRIC` array contains the following quantities:

\begin{align}
{\rm ALPHA} &:= \alpha\ ,\\
{\rm BETAU} &:= \beta^{i}\ ,\\
{\rm GDD}   &:= \gamma_{ij}\ ,\\
{\rm GUPDD} &:= \gamma^{ij}\ ,\\
{\rm SQRTGAMMA} &:= \sqrt{\gamma} \ ,\ \gamma \equiv \det\left(\gamma_{ij}\right)\ .
\end{align}

We start by declaring a function that sets up these variables.

In [2]:
# Step 1: Declare basic ADM variables to be used by IllinoisGRMHD
# Step 1.a: Set spatial dimension to 3
DIM = 3

# Step 1.b: Set up alpha
alpha = sp.symbols("ADM_3METRIC[ALPHA]",real=True)

# Step 1.b: Set up beta^{i}
betaU   = ixp.zerorank2()
for i in range(DIM):
    betaU[i] = sp.symbols('ADM_3METRIC[SHIFT'+chr(ord('X')+i)+"]",real=True)

# Step 1.c: Set up gamma_{ij}
gammaDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(i,DIM):
        gammaDD[i][j] = gammaDD[j][i] = sp.symbols('ADM_3METRIC[G'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)

# Step 1.d: Set up gamma^{ij}
gammaUU = ixp.zerorank2()
for i in range(3):
    for j in range(i,3):
        gammaUU[i][j] = gammaUU[j][i] = sp.symbols('ADM_3METRIC[GUP'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)
        
# Step 1.e: Set up sqrt(gamma)
sqrtgamma = sp.symbols("ADM_3METRIC[SQRTGAMMA]",real=True)

<a id='compute_gamma_inv_and_sqrtgamma'></a>

## Step 1.a: The `compute__ADM_gammaUU_and_sqrtgamma __in_terms_of__ADM_3METRIC.h` file \[Back to [top](#toc)\]
$$\label{compute_gamma_inv_and_sqrtgamma}$$

Now we compute the inverse ADM 3-metric $\gamma^{ij}$ from $\gamma_{ij}$, by inverting $\gamma_{ij}$ using our `symm_matrix_inverter3x3()` function from the [indexedexp.py](/edit/NRPyIGM/indexedexp.py) NRPy+ module.

In [3]:
# Step 1.a: The compute_ADM_gammaUU_and_sqrtgamma.h file
# Step 1.a.i: Compute eh inverse ADM 3-metric and the determinant of the ADM 3-metric
gammaINV, gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)

# Step 1.a.ii: Populate the ADM_3METRIC[GUPUU] and ADM_3METRIC[SQRTGAMMA]
#           with the results of our inverter
string = outputC([ gammaINV[0][0],       gammaINV[0][1],       gammaINV[0][2],
                   gammaINV[1][1],       gammaINV[1][2],       gammaINV[2][2], sp.sqrt(gammaDET)],
        ["ADM_3METRIC[GAMMAUPXX]","ADM_3METRIC[GAMMAUPXY]","ADM_3METRIC[GAMMAUPXZ]",
         "ADM_3METRIC[GAMMAUPYY]","ADM_3METRIC[GAMMAUPYZ]","ADM_3METRIC[GAMMAUPZZ]", "ADM_3METRIC[SQRTGAMMA]"],
        filename="returnstring", params="outCverbose=False")

# Step 1.a.iii: Replace pow(blah, 2) with (blah)*(blah)
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2

# Step 1.a.iv: Output result to file
filename = "compute__ADM_gammaUU_and_sqrtgamma__in_terms_of__ADM_3METRIC.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)
    
print("Just generated the file: "+filepath)

Just generated the file: ../src/NRPy_generated_headers/compute__ADM_gammaUU_and_sqrtgamma__in_terms_of__ADM_3METRIC.h


<a id='compute_g4dd'></a>

## Step 1.b: The `compute__g4DD__in_terms_of__ADM_3METRIC.h` file \[Back to [top](#toc)\]
$$\label{compute_g4dd}$$

We now compute the ADM 4-metric, $g_{\mu\nu}$, given by (see equation 2.122 in [Baumgarte & Shapiro's Numerical Relativity](https://www.google.com/books/edition/Numerical_Relativity/dxU1OEinvRUC?hl=en&gbpv=0))

$$
g_{\mu\nu}
=
\begin{pmatrix}
-\alpha^{2} + \beta_{\ell}\beta^{\ell} & \beta_{i}\\
\beta_{j} & \gamma_{ij}
\end{pmatrix}\ .
$$

We do this with using the [BSSN/ADMBSSN_tofrom_4metric](/edit/NRPyIGM/BSSN/ADMBSSN_tofrom_4metric.py) NRPy+ module.

In [4]:
# Step 1.b: Compute the ADM 4-metric, g_{\mu\nu}
# Step 1.b.i: Load the BSSN.ADMBSSN_tofrom_4metric NRPy+ module
import BSSN.ADMBSSN_tofrom_4metric as AB4m

# Step 1.b.ii: Compute the g_{\mu\nu} in terms of our ADM variables
AB4m.g4DD_ito_BSSN_or_ADM("ADM",gammaDD=gammaDD,betaU=betaU,alpha=alpha)

# Step 1.b.iii: Set up lists to store the expressions and output variables
exprlist = []
namelist = []
for mu in range(4):
    for nu in range(4):
        exprlist.append(AB4m.g4DD[mu][nu])
        namelist.append("g4dn["+str(mu)+"]["+str(nu)+"]")
        
# Step 1.b.iv: Convert our results to C output
string = outputC(exprlist,namelist,"returnstring", params="outCverbose=False")
# Step 1.b.v: Replace pow(blah, 2) with (blah)*(blah)
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2

# Step 1.b.vi: Output to file
filename = "compute__g4DD__in_terms_of__ADM_3METRIC.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)

print("Just generated the file: "+filepath)

Just generated the file: ../src/NRPy_generated_headers/compute__g4DD__in_terms_of__ADM_3METRIC.h


<a id='compute_g4uu'></a>

## Step 1.c: The `compute__g4UU__in_terms_of__ADM_3METRIC.h` file \[Back to [top](#toc)\]
$$\label{compute_g4uu}$$

We now compute the inverse ADM 4-metric, $g^{\mu\nu}$, given by (see equation 2.119 in [Baumgarte & Shapiro's Numerical Relativity](https://www.google.com/books/edition/Numerical_Relativity/dxU1OEinvRUC?hl=en&gbpv=0))

$$
g^{\mu\nu}
=
\begin{pmatrix}
-\alpha^{-2} & \alpha^{-2}\beta^{i}\\
\alpha^{-2}\beta^{j} & \gamma^{ij} - \alpha^{-2}\beta^i\beta^{j}
\end{pmatrix}\ .
$$

We do this with using the [BSSN/ADMBSSN_tofrom_4metric](/edit/NRPyIGM/BSSN/ADMBSSN_tofrom_4metric.py) NRPy+ module.

In [5]:
# Step 1.c: Compute the inverse ADM 4-metric, g^{\mu\nu}
# Step 1.c.i: Load the BSSN.ADMBSSN_tofrom_4metric NRPy+ module
import BSSN.ADMBSSN_tofrom_4metric as AB4m

# Step 1.c.ii: Compute the g_{\mu\nu} in terms of our ADM variables
AB4m.g4UU_ito_BSSN_or_ADM("ADM",betaU=betaU,alpha=alpha,gammaUU=gammaUU)

# Step 1.c.iii: Set up lists to store the expressions and output variables
exprlist = []
namelist = []
for mu in range(4):
    for nu in range(4):
        exprlist.append(AB4m.g4UU[mu][nu])
        namelist.append("g4up["+str(mu)+"]["+str(nu)+"]")
        
# Step 1.c.iv: Convert our results to C output
string = outputC(exprlist,namelist,"returnstring", params="outCverbose=False")
# Step 1.c.v: Replace pow(blah, 2) with (blah)*(blah)
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2

# Step 1.c.vi: Output to file
filename = "compute__g4UU__in_terms_of__ADM_3METRIC.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)

print("Just generated the file: "+filepath)

Just generated the file: ../src/NRPy_generated_headers/compute__g4UU__in_terms_of__ADM_3METRIC.h


<a id='conf_metric_vars'></a>

# Step 2: The `CONF_METRIC` variable declaration \[Back to [top](#toc)\]
$$\label{conf_metric_vars}$$

The `CONF_METRIC` array contains the following quantities:

\begin{align}
{\rm PHI} &:= \phi\ ,\ \text{conformal factor}\ ,\\
{\rm GAMMADD} &:= \bar\gamma_{ij}\ ,\\
{\rm GAMMAUU} &:= \bar\gamma^{ij}\ .
\end{align}

We start by declaring a function that sets up these variables.

In [6]:
# Step 2: Set the the conformal metric variables to be used by IllinoisGRMHD
# Step 2.a: Set up phi
cf_phi = sp.symbols("CONF_METRIC[PHI]",real=True)

# Step 2.b: Set up \bar\gamma_{ij}
gammabarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(i,DIM):
        gammabarDD[i][j] = gammabarDD[j][i] = sp.symbols('CONF_METRIC[GAMMA'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)

# Step 2.c: Set up \bar\gamma^{ij}
gammabarUU = ixp.zerorank2()
for i in range(3):
    for j in range(i,3):
        gammabarUU[i][j] = gammabarUU[j][i] = sp.symbols('CONF_METRIC[GAMMAUP'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)

<a id='compute__lapseinv_psi2_psi4_psi6'></a>

## Step 2.a: The `compute__lapseinv_psi2_psi4_psi6__in_terms_of __ADM_3METRIC_and_CONF_METRIC.h` file \[Back to [top](#toc)\]
$$\label{compute__lapseinv_psi2_psi4_psi6}$$

We now compute basic auxiliary quantities, specifically $\alpha^{-1}$, $\psi^{2}$, $\psi^{4}$, and $\psi^{6}$.

In [7]:
# Step 2.a: Compute useful auxiliary quantities
# Step 2.a.i: Set up lapseinv = 1/alpha
lapseinv = 1.0/alpha

# Step 2.a.ii: Set up psi2 = psi^{2} = exp(2*phi)
psi2 = sp.exp(2*cf_phi)

# Step 2.a.iii: Set up psi4 = psi^{4} = psi^{2} * psi^{2}
psi4 = sp.symbols("psi2",real=True)*sp.symbols("psi2",real=True)

# Step 2.a.iv: Set up psi6 = psi^{6} = psi^{2} * psi^{4}
psi6 = sp.symbols("psi2",real=True)*sp.symbols("psi4",real=True)

# Step 2.a.v: Convert our results to C output
string = outputC([ lapseinv , psi2 , psi4 , psi6],
                 ["CCTK_REAL lapseinv","CCTK_REAL psi2","CCTK_REAL psi4","CCTK_REAL psi6"],"returnstring", params="outCverbose=False")

# Step 2.a.vi: Replace pow(blah, 2) with (blah)*(blah)
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2

# Step 2.a.vii: Output to file
filename = "compute__lapseinv_psi2_psi4_psi6__in_terms_of__ADM_3METRIC_and_CONF_METRIC.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)
    
print("Just generated the file: "+filepath)

Just generated the file: ../src/NRPy_generated_headers/compute__lapseinv_psi2_psi4_psi6__in_terms_of__ADM_3METRIC_and_CONF_METRIC.h


<a id='read_in_conf_metric_vars'></a>

## Step 2.b: The `VARS_FOR_METRIC_FACEVALS.h` and  `read_in_CONF_METRIC_from_gridfunctions.h` files \[Back to [top](#toc)\]
$$\label{read_in_conf_metric_vars}$$

We now create a header files to substitute the following pieces of code:

1. In `IllinoisGRMHD_headers.h`:

```c
// The order here MATTERS, as we assume that GAMMAUPXX+1=GAMMAUPYY, etc.
static const int PHI=0,PSI=1,GAMMATILDEXX=2,GAMMATILDEXY=3,GAMMATILDEXZ=4,GAMMATILDEYY=5,GAMMATILDEYZ=6,GAMMATILDEZZ=7,
  LAPM1=8,SHIFTX=9,SHIFTY=10,SHIFTZ=11,GAMMATILDEUPXX=12,GAMMATILDEUPYY=13,GAMMATILDEUPZZ=14,
  NUMVARS_FOR_METRIC_FACEVALS=15; //<-- Be _sure_ to set this correctly, or you'll have memory access bugs!

// These are not used for facevals in the reconstruction step, but boy are they useful anyway. 
static const int GAMMAUPXY=15,GAMMAUPXZ=16,GAMMAUPYZ=17,
  NUMVARS_FOR_METRIC=18; //<-- Be _sure_ to set this correctly, or you'll have memory access bugs!
```
2. In `driver_evaluate_MHD_rhs.C`, `outer_boundaries.C`, `driver_conserv_to_prims.C`, and `set_IllinoisGRMHD_metric_GRMHD_variables_based_on_HydroBase_and_ADMBase_variables.C`, the last one being part of the `ID_converter_ILGRMHD` ETK thorn:

```c
  METRIC[ww]=phi_bssn;ww++;
  METRIC[ww]=psi_bssn;ww++;
  METRIC[ww]=gtxx;    ww++;
  METRIC[ww]=gtxy;    ww++;
  METRIC[ww]=gtxz;    ww++;
  METRIC[ww]=gtyy;    ww++;
  METRIC[ww]=gtyz;    ww++;
  METRIC[ww]=gtzz;    ww++;
  METRIC[ww]=lapm1;   ww++;
  METRIC[ww]=betax;   ww++;
  METRIC[ww]=betay;   ww++;
  METRIC[ww]=betaz;   ww++;
  METRIC[ww]=gtupxx;  ww++;
  METRIC[ww]=gtupyy;  ww++;
  METRIC[ww]=gtupzz;  ww++;
```

For starters, we will replace `METRIC` by `CONF_METRIC`.

In [8]:
# Set INDEXNAME and GFNAME
INDEXNAME = int(0)
GFNAME    = int(1)

# Start setting up the gridfunction indices with phi and psi
gfslist = [["CM_PHI","phi_bssn"],["CM_PSI","psi_bssn"]]

# Add the indices for \tilde{\gamma}_{ij}
gfslist.append(["CM_GAMMATILDEXX","gtxx"])
gfslist.append(["CM_GAMMATILDEXY","gtxy"])
gfslist.append(["CM_GAMMATILDEXZ","gtxz"])
gfslist.append(["CM_GAMMATILDEYY","gtyy"])
gfslist.append(["CM_GAMMATILDEYZ","gtyz"])
gfslist.append(["CM_GAMMATILDEZZ","gtzz"])

# Add alpha, \beta^{i}, and \tilde{\gamma}^{ii}
gfslist.append(["CM_LAPM1","lapm1"])
gfslist.append(["CM_SHIFTX","betax"])
gfslist.append(["CM_SHIFTY","betay"])
gfslist.append(["CM_SHIFTZ","betaz"])
gfslist.append(["CM_GAMMATILDEUPXX","gtxx"])
gfslist.append(["CM_GAMMATILDEUPYY","gtyy"])
gfslist.append(["CM_GAMMATILDEUPZZ","gtzz"])

othervars =     [["CM_GAMMATILDEUPXY","gtxy"]]
othervars.append(["CM_GAMMATILDEUPXZ","gtxy"])
othervars.append(["CM_GAMMATILDEUPYZ","gtxy"])

# Define the indices in the VARS_FOR_METRIC_FACEVALS.h header file
# Start with the variables used in the face value reconstructions
string = "/* Variables used for face value reconstructions */\n"
for j in range(len(gfslist)):
    string += "static const int "+gfslist[j][INDEXNAME]
    # This is purely cosmetic, to keep the equal signs aligned
    for k in range(len("NUMVARS_FOR_CONF_METRIC_FACEVALS") - len(gfslist[j][INDEXNAME])):
        string += " "
    string += " = "+str(j)+";\n"
string += "static const int NUMVARS_FOR_CONF_METRIC_FACEVALS = "+str(len(gfslist))+";\n\n"

# Then set other useful variables (i.e. off-diagonal metric components)
string += "/* Other useful variables */\n"
for j in range(len(othervars)):
    string += "static const int "+othervars[j][INDEXNAME]
    for k in range(len("NUMVARS_FOR_CONF_METRIC_FACEVALS") - len(othervars[j][INDEXNAME])):
        string += " "
    string += " = "+str(j+len(gfslist))+";\n"
string += "static const int NUMVARS_FOR_CONF_METRIC          = "+str(len(gfslist)+len(othervars))+";\n\n"

# Write string to file
filename = "VARS_FOR_CONF_METRIC_FACEVALS.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)
print("Just generated the file: "+filepath)

# Set up the read_in_CONF_METRIC_variables.h file
string = "/* Reading in conformal metric variables from gridfunctions */\n"
for j in range(len(gfslist)):
    string += "CONF_METRIC["+gfslist[j][INDEXNAME]+"]"
    for k in range(len("CM_GAMMATILDEUPXX") - len(gfslist[j][INDEXNAME])):
        string += " "
    string += " = "+gfslist[j][GFNAME]+";\n"
    
# Write string to file
filename = "read_in_CONF_METRIC_from_gridfunctions.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)
print("Just generated the file: "+filepath)

Just generated the file: ../src/NRPy_generated_headers/VARS_FOR_CONF_METRIC_FACEVALS.h
Just generated the file: ../src/NRPy_generated_headers/read_in_CONF_METRIC_from_gridfunctions.h


<a id='tmunu'></a>

# Step 3: Computing the energy-momentum tensor \[Back to [top](#toc)\]
$$\label{tmunu}$$

The energy-momentum tensor for general relativistic magnetohydrodynamics (GRMHD) is given by

$$
T^{\mu\nu}_{\rm GRMHD} = T^{\mu\nu}_{\rm GRHD} + T^{\mu\nu}_{\rm EM}\ ,
$$

where

$$
T^{\mu\nu}_{\rm GRHD} = h\rho_{b}u^{\mu}u^{\nu} + Pg^{\mu\nu}\ ,
$$

is the general relativistic hydrodynamics (GRHD) energy-momentum tensor and 

$$
T^{\mu\nu}_{\rm EM} = b^2 u^{\mu} u^{\nu} + \frac{b^2}{2} g^{\mu\nu} + b^\mu b^\nu\ ,
$$

is the electromagnetic energy-momentum tensors, respectively. Here, $u^{\mu}$ is the fluid's 4-velocity and

\begin{align}
\sqrt{4\pi} b^0 = B^0_{\rm (u)} &= \frac{u_j B^j}{\alpha}\ ,\\
\sqrt{4\pi} b^i = B^i_{\rm (u)} &= \frac{B^i + (u_j B^j) u^i}{\alpha u^0}\ ,
\end{align}

with $b^{2}\equiv g_{\mu\nu}b^{\mu}b^{\nu}$.

<a id='tmunu_grhd'></a>

## Step 3.a: Computing the GRHD energy-momentum tensor \[Back to [top](#toc)\]
$$\label{tmunu_grhd}$$

$\newcommand{\Pcold}{P_{\text{cold}}}$
$\newcommand{\epscold}{\epsilon_{\text{cold}}}$
$\newcommand{\epsth}{\epsilon_{\text{th}}}$
$\newcommand{\Gammath}{\Gamma_{\text{th}}}$
$\newcommand{\rhob}{\rho_{b}}$

In order to compute the GRHD energy-momentum tensor, we will need the following quantities:

1. The enthalpy, $h$
1. The baryonic density, $\rho_{b}$
1. the 4-velocity, $u^{\mu}$
1. The pressure, $P$

The enthalpy is given by

$$
h = 1 + \epsilon + \frac{P}{\rhob}\ .
$$

For the hybrid EOS used by `IllinoisGRMHD`, namely

$$
P(\rho_{b},\epsilon) = \Pcold(\rhob) + \left(\Gammath-1\right)\rhob\left(\epsilon - \epscold\right)\ ,
$$

we have

$$
\boxed{\epsilon = \epscold + \frac{P-\Pcold}{\left(\Gammath-1\right)\rhob}}\ .
$$

To compute the 4-velocity $u^{\mu}$, we will use the `u4U_in_terms_of_vU_apply_speed_limit()` function from the [GRHD.equations NRPy+ module](/edit/NRPyIGM/GRHD/equations.py). Finally, the GRHD energy-momentum tensor is computed using the `compute_T4UU()` function, from the same module.

In [9]:
# Step 3: Computing T^{\mu\nu}_{GRMHD} and T_{\mu\nu}^{GRMHD}
# Step 3.a: Computing T^{\mu\nu}_{GRHD}
# Step 3.a.i: Import the GRHD NRPy+ module
import GRHD.equations as GRHD

# Step 3.a.ii: Declare needed variables, named according to IllinoisGRMHD
rho_b, P, P_cold, eps_cold, Gamma_th = sp.symbols("U[RHOB] U[PRESSURE] P_cold eps_cold Gamma_th",real=True)

# Step 3.a.iii: Compute epsilon
epsilon = eps_cold + (P - P_cold)/(rho_b * (Gamma_th - 1))

# Step 3.a.iv: Compute u^{\mu} from v^{i}
vU = [sp.symbols("U[VX]",real=True),sp.symbols("U[VY]",real=True),sp.symbols("U[VZ]",real=True)]
GRHD.u4U_in_terms_of_vU_apply_speed_limit(alpha,betaU,gammaDD, vU)
u4U = GRHD.u4_ito_3velsU

# Step 3.a.v: Compute the energy-momentum tensor
GRHD.compute_T4UU(gammaDD,betaU,alpha, rho_b,P,epsilon,u4U)

<a id='tmunu_em'></a>

## Step 3.b: Computing the EM energy-momentum tensor \[Back to [top](#toc)\]
$$\label{tmunu_em}$$

In order to compute the EM energy-momentum tensor, we will need to compute $b^{\mu}$ and $b^{2}$. We then compute the EM energy-momentum tensor using the `compute_TEM4UU()` from the [GRFFE.equations NRPy+ module](/edit/NRPyIGM/GRFFE/equations.py)

In [10]:
# Step 3.b: Computing T^{\mu\nu}_{EM}
# Step 3.b.i: Import the GRFFE NRPy+ module
import GRFFE.equations as GRFFE

# Step 3.b.ii: Declare necessary variables, named according to IllinoisGRMHD
smallb4U = [sp.symbols("smallb[SMALLBT]",real=True), \
            sp.symbols("smallb[SMALLBX]",real=True), \
            sp.symbols("smallb[SMALLBY]",real=True), \
            sp.symbols("smallb[SMALLBZ]",real=True)]

# Step 3.b.iii: Compute b^{2} = g_{\mu\nu}b^{\mu}b^{\nu}
smallbsquared = sp.sympify(0)
for mu in range(4):
    for nu in range(4):
        smallbsquared += AB4m.g4DD[mu][nu] * smallb4U[mu] * smallb4U[nu]

# Step 3.b.iv: Compute the EM energy-momentum tensor
GRFFE.compute_TEM4UU(gammaDD,betaU,alpha, smallb4U, smallbsquared,u4U)

<a id='tmunu_tupmunu'></a>

## Step 3.c: The `compute__TUPMUNU.h` file \[Back to [top](#toc)\]
$$\label{tmunu_tupmunu}$$

Now that we have $T^{\mu\nu}_{\rm GRHD}$ and $T^{\mu\nu}_{\rm EM}$, we can easily compute the GRMHD energy-momentum tensor

$$
\boxed{T^{\mu\nu}_{\rm GRMHD} = T^{\mu\nu}_{\rm GRHD} + T^{\mu\nu}_{\rm EM}}\ .
$$

In [11]:
# Step 3.c: The compute_TUPMUNU.h file
# Step 3.c.i: Compute T^{\mu\nu}_{GRMHD}
TGRMHD4UU = GRHD.T4UU + GRFFE.TEM4UU

# Step 3.c.ii: Prepare T^{\mu\nu}_{GRMHD} for C code output
TIGM4UU = []
for mu in range(4):
    for nu in range(4):
        TIGM4UU.append(TGRMHD4UU[mu][nu])

# Step 3.c.iii: Declare the variables expected by IllinoisGRMHD
TUPMUNU = ["TUPMUNU["+str(count)+"]" for count in range(16)]

# Step 3.c.iv: Convert our results to C output
string = outputC(TIGM4UU,TUPMUNU,"returnstring", params="outCverbose=False")

# Step 3.c.v: Replace pow(blah, 2) with (blah)*(blah)
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2

# Step 3.c.vi: Output to file
filename = "compute__TUPMUNU.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)
print("Just generated the file: "+filepath)

Just generated the file: ../src/NRPy_generated_headers/compute__TUPMUNU.h


<a id='tmunu_tdnmunu'></a>

## Step 3.d: The `compute__TDNMUNU.h` file \[Back to [top](#toc)\]
$$\label{tmunu_tdnmunu}$$

Now we compute 

$$
\boxed{T_{\mu\nu}^{\rm GRMHD} = g_{\mu\rho}g_{\nu\sigma}T^{\rho\sigma}_{\rm GRMHD}}\ .
$$

In [12]:
# Step 3.d: The compute_TDNMUNU.h file
# Step 3.d.i: Compute T_{\mu\nu}^{GRMHD}
TGRMHD4DD = ixp.zerorank2(DIM=4)
for mu in range(4):
    for nu in range(4):
        for rho in range(4):
            for sigma in range(4):
                TGRMHD4DD[mu][nu] += AB4m.g4DD[mu][rho] * AB4m.g4DD[nu][sigma] * TGRMHD4UU[rho][sigma]

# Step 3.d.ii: Prepare T^{\mu\nu}_{GRMHD} for C code output
TIGM4DD = []
for mu in range(4):
    for nu in range(4):
        TIGM4DD.append(TGRMHD4DD[mu][nu])

# Step 3.d.iii: Declare the variables expected by IllinoisGRMHD
TDNMUNU = ["TDNMUNU["+str(count)+"]" for count in range(16)]

# Step 3.d.iv: Convert our results to C output
string = outputC(TIGM4DD,TDNMUNU,"returnstring", params="outCverbose=False")

# Step 3.d.v: Replace pow(blah, 2) with (blah)*(blah)
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2

# Step 3.d.vi: Output to file
filename = "compute__TDNMUNU.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)
print("Just generated the file: "+filepath)

Just generated the file: ../src/NRPy_generated_headers/compute__TDNMUNU.h


<a id='read_in_conf_metric_vars'></a>

## Step 4: The `GRMHD_VARS.h` and  `read_IN_PRIMS_and_OUT_PRIMS_from_gridfunctions.h` files \[Back to [top](#toc)\]
$$\label{read_in_conf_metric_vars}$$

We now write codes to substitute:

1. In `IllinoisGRMHD_headers.h`:
```c
// The order here MATTERS, and must be consistent with the order in the IN_PRIMS[] array in driver_evaluate_MHD_rhs.C.
static const int RHOB=0,PRESSURE=1,VX=2,VY=3,VZ=4,
  BX_CENTER=5,BY_CENTER=6,BZ_CENTER=7,BX_STAGGER=8,BY_STAGGER=9,BZ_STAGGER=10,
  VXR=11,VYR=12,VZR=13,VXL=14,VYL=15,VZL=16,MAXNUMVARS=17;  //<-- Be _sure_ to define MAXNUMVARS appropriately!
static const int UT=0,UX=1,UY=2,UZ=3;
```

2. In `driver_evaluate_MHD_rhs.C`:
```c
int ww=0;
IN_PRIMS[ww].gf=rho_b;      OUT_PRIMS_R[ww].gf=rho_br;      OUT_PRIMS_L[ww].gf=rho_bl;      ww++;
IN_PRIMS[ww].gf=P;          OUT_PRIMS_R[ww].gf=Pr;          OUT_PRIMS_L[ww].gf=Pl;          ww++;
IN_PRIMS[ww].gf=vx;         OUT_PRIMS_R[ww].gf=vxr;         OUT_PRIMS_L[ww].gf=vxl;         ww++;
IN_PRIMS[ww].gf=vy;         OUT_PRIMS_R[ww].gf=vyr;         OUT_PRIMS_L[ww].gf=vyl;         ww++;
IN_PRIMS[ww].gf=vz;         OUT_PRIMS_R[ww].gf=vzr;         OUT_PRIMS_L[ww].gf=vzl;         ww++;
IN_PRIMS[ww].gf=Bx;         OUT_PRIMS_R[ww].gf=Bxr;         OUT_PRIMS_L[ww].gf=Bxl;         ww++;
IN_PRIMS[ww].gf=By;         OUT_PRIMS_R[ww].gf=Byr;         OUT_PRIMS_L[ww].gf=Byl;         ww++;
IN_PRIMS[ww].gf=Bz;         OUT_PRIMS_R[ww].gf=Bzr;         OUT_PRIMS_L[ww].gf=Bzl;         ww++;
IN_PRIMS[ww].gf=Bx_stagger; OUT_PRIMS_R[ww].gf=Bx_staggerr; OUT_PRIMS_L[ww].gf=Bx_staggerl; ww++;
IN_PRIMS[ww].gf=By_stagger; OUT_PRIMS_R[ww].gf=By_staggerr; OUT_PRIMS_L[ww].gf=By_staggerl; ww++;
IN_PRIMS[ww].gf=Bz_stagger; OUT_PRIMS_R[ww].gf=Bz_staggerr; OUT_PRIMS_L[ww].gf=Bz_staggerl; ww++;
IN_PRIMS[ww].gf=vxr;        OUT_PRIMS_R[ww].gf=vxrr;        OUT_PRIMS_L[ww].gf=vxrl;        ww++;
IN_PRIMS[ww].gf=vyr;        OUT_PRIMS_R[ww].gf=vyrr;        OUT_PRIMS_L[ww].gf=vyrl;        ww++;
IN_PRIMS[ww].gf=vzr;        OUT_PRIMS_R[ww].gf=vzrr;        OUT_PRIMS_L[ww].gf=vzrl;        ww++;
IN_PRIMS[ww].gf=vxl;        OUT_PRIMS_R[ww].gf=vxlr;        OUT_PRIMS_L[ww].gf=vxll;        ww++;
IN_PRIMS[ww].gf=vyl;        OUT_PRIMS_R[ww].gf=vylr;        OUT_PRIMS_L[ww].gf=vyll;        ww++;
IN_PRIMS[ww].gf=vzl;        OUT_PRIMS_R[ww].gf=vzlr;        OUT_PRIMS_L[ww].gf=vzll;        ww++;
```

In [13]:
# Set INDEXNAME, GFNAME, GFNAME_R, and GFNAME_L
INDEXNAME = int(0)
GFNAME    = int(1)
GFNAME_R  = int(2)
GFNAME_L  = int(3)

# Add indices and variables
# First for rho_b and the pressure
gfslist =     [["RHOB","rho_b","rho_br","rho_bl"]]
gfslist.append(["PRESSURE","P","Pr","Pl"])

# Second, the 3-velocity
gfslist.append(["VX","vx","vxr","vxl"])
gfslist.append(["VY","vy","vyr","vyl"])
gfslist.append(["VZ","vz","vzr","vzl"])

# Next, the unstaggered and staggered magnetic field
gfslist.append(["BX_CENTER","Bx","Bxr","Bxl"])
gfslist.append(["BY_CENTER","By","Byr","Byl"])
gfslist.append(["BZ_CENTER","Bz","Bzr","Bzl"])
gfslist.append(["BX_STAGGER","Bx_stagger","Bx_staggerr","Bx_staggerl"])
gfslist.append(["BY_STAGGER","By_stagger","By_staggerr","By_staggerl"])
gfslist.append(["BZ_STAGGER","Bz_stagger","Bz_staggerr","Bz_staggerl"])

# Then, the right/left values of the 3-velocity
gfslist.append(["VXR","vxr","vxrr","vxrl"])
gfslist.append(["VYR","vyr","vyrr","vyrl"])
gfslist.append(["VZR","vzr","vzrr","vzrl"])
gfslist.append(["VXL","vxl","vxlr","vxll"])
gfslist.append(["VYL","vyl","vylr","vyll"])
gfslist.append(["VZL","vzl","vzlr","vzll"])

# Finally, the 4-velocity
u4list = ["UT","UX","UY","UZ"]

# Define the indices in the GRMHD_VARS.h header file
# Start with the variables used in the face value reconstructions
string = "/* GRMHD variables */\n"
for j in range(len(gfslist)):
    string += "static const int "+gfslist[j][INDEXNAME]
    # This is purely cosmetic, to keep the equal signs aligned
    for k in range(len("BX_STAGGER") - len(gfslist[j][INDEXNAME])):
        string += " "
    string += " = "+str(j)+";\n"
string += "static const int MAXNUMVARS = "+str(len(gfslist))+";\n\n"

string += "/* 4-velocity */\n"
for mu in range(4):
    string += "static const int "+u4list[mu]
    for k in range(len("BX_STAGGER") - len(u4list[mu])):
        string += " "
    string += " = "+str(mu)+";\n"

# Write string to file
filename = "GRMHD_VARS.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)
print("Just generated the file: "+filepath)

# Set up the read_in_CONF_METRIC_variables.h file
string = "/* Reading in IN_PRIMS, OUT_PRIMS_R, and OUT_PRIMS_L */\n"
for j in range(len(gfslist)):
    # Set the IN_PRIMS reading
    string += "IN_PRIMS["+gfslist[j][INDEXNAME]+"].gf"
    for k in range(len("BX_STAGGER") - len(gfslist[j][INDEXNAME])):
        string += " "
    string += " = "+gfslist[j][GFNAME]+"; "
    for k in range(len("Bx_stagger") - len(gfslist[j][GFNAME])):
        string += " "
    
    # Set the OUT_PRIMS_R reading
    string += "OUT_PRIMS_R["+gfslist[j][INDEXNAME]+"].gf"
    for k in range(len("BX_STAGGER") - len(gfslist[j][INDEXNAME])):
        string += " "
    string += " = "+gfslist[j][GFNAME_R]+"; "
    for k in range(len("Bx_staggerr") - len(gfslist[j][GFNAME_R])):
        string += " "
        
    # Set the OUT_PRIMS_L reading
    string += "OUT_PRIMS_L["+gfslist[j][INDEXNAME]+"].gf"
    for k in range(len("BX_STAGGER") - len(gfslist[j][INDEXNAME])):
        string += " "
    string += " = "+gfslist[j][GFNAME_L]+";"
    for k in range(len("Bx_staggerl") - len(gfslist[j][GFNAME_L])):
        string += " "
    string += "\n"

# Write string to file
filename = "read_in_GRMHD_from_gridfunctions.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)
print("Just generated the file: "+filepath)

Just generated the file: ../src/NRPy_generated_headers/GRMHD_VARS.h
Just generated the file: ../src/NRPy_generated_headers/read_in_GRMHD_from_gridfunctions.h


<a id='read_in_interp_vars'></a>

## Step 5: The `INTERP_VARS.h` and  `read_in_INTERP_VARS_from_gridfunctions.h` files \[Back to [top](#toc)\]
$$\label{read_in_interp_vars}$$

1. In `IllinoisGRMHD_headers.h`:
```c
// The "I" suffix denotes interpolation. In other words, these
//    definitions are used for interpolation ONLY. The order here
//    matters as well!
static const int SHIFTXI=0,SHIFTYI=1,SHIFTZI=2,GAMMAUPXXI=3,GAMMAUPXYI=4,GAMMAUPXZI=5,GAMMAUPYYI=6,GAMMAUPYZI=7,GAMMAUPZZI=8,
  PSII=9,LAPM1I=10,A_XI=11,A_YI=12,A_ZI=13,LAPSE_PSI2I=14,LAPSE_OVER_PSI6I=15,MAXNUMINTERP=16;
```

2. In `driver_evaluate_MHD_rhs.C`:
```c
  ww=0;
  interp_vars[ww]=betax;   ww++;
  interp_vars[ww]=betay;   ww++;
  interp_vars[ww]=betaz;   ww++;
  interp_vars[ww]=gtupxx;  ww++;
  interp_vars[ww]=gtupxy;  ww++;
  interp_vars[ww]=gtupxz;  ww++;
  interp_vars[ww]=gtupyy;  ww++;
  interp_vars[ww]=gtupyz;  ww++;
  interp_vars[ww]=gtupzz;  ww++;
  interp_vars[ww]=psi_bssn;ww++;
  interp_vars[ww]=lapm1;   ww++;
  interp_vars[ww]=Ax;      ww++;
  interp_vars[ww]=Ay;      ww++;
  interp_vars[ww]=Az;      ww++;
  int max_num_interp_variables=ww;
```

In [14]:
# Set INDEXNAME and GFNAME
INDEXNAME = int(0)
GFNAME    = int(1)

# Shift vector, beta^{i}
gfslist =     [["INTERP_SHIFTX","betax"]]
gfslist.append(["INTERP_SHIFTY","betay"])
gfslist.append(["INTERP_SHIFTZ","betaz"])

# Conformal inverse metric, \tilde{\gamma}^{ij}
gfslist.append(["INTERP_GAMMATILDEUPXX","gtupxx"])
gfslist.append(["INTERP_GAMMATILDEUPXY","gtupxy"])
gfslist.append(["INTERP_GAMMATILDEUPXZ","gtupxz"])
gfslist.append(["INTERP_GAMMATILDEUPYY","gtupyy"])
gfslist.append(["INTERP_GAMMATILDEUPYZ","gtupyz"])
gfslist.append(["INTERP_GAMMATILDEUPZZ","gtupzz"])

# psi and alpha-1
gfslist.append(["INTERP_PSI","psi_bssn"])
gfslist.append(["INTERP_LAPM1","lapm1"])

# A fields
gfslist.append(["INTERP_AX","Ax"])
gfslist.append(["INTERP_AY","Ay"])
gfslist.append(["INTERP_AZ","Az"])

# Auxiliary quantities
gfslist.append(["INTERP_LAPSE_PSI2"])
gfslist.append(["INTERP_LAPSE_OVER_PSI6"])

# Define the indices in the GRMHD_VARS.h header file
# Start with the variables used in the face value reconstructions
string = "/* Interpolation variables */\n"
for j in range(len(gfslist)):
    string += "static const int "+gfslist[j][INDEXNAME]
    # This is purely cosmetic, to keep the equal signs aligned
    for k in range(len("INTERP_LAPSE_OVER_PSI6") - len(gfslist[j][INDEXNAME])):
        string += " "
    string += " = "+str(j)+";\n"
string += "static const int MAXNUMINTERP           = "+str(len(gfslist))+";\n"

# Write string to file
filename = "INTERP_VARS.h"
filepath = os.path.join(NRPy_headers_dir_path,filename)
NRPy_IGM_write_to_file(filepath,filename,string)
print("Just generated the file: "+filepath)

Just generated the file: ../src/NRPy_generated_headers/INTERP_VARS.h


<a id='latex_pdf_output'></a>

# Step n: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__A_i_rhs_no_gauge_terms.pdf](Tutorial-IllinoisGRMHD__A_i_rhs_no_gauge_terms.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [15]:
latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.tex
!rm -f Tut*.out Tut*.aux Tut*.log